# LUME-services

### Configuration:

Set the following environment variables:

```
export LUME_MODEL_DB__HOST=127.0.0.1
export LUME_MODEL_DB__USER=root
export LUME_MODEL_DB__PASSWORD=test
export LUME_MODEL_DB__PORT=3306
export LUME_MODEL_DB__DATABASE=model_db
export LUME_MODEL_DB__CONNECTION__POOL_SIZE=1
export LUME_PREFECT__SERVER__TAG=core-1.2.4
export LUME_PREFECT__SERVER__HOST_PORT=4200
export LUME_PREFECT__SERVER__HOST=http://localhost
export LUME_PREFECT__AGENT__HOST_PORT=5000
export LUME_PREFECT__AGENT__HOST=http://localhost
export LUME_RESULTS_DB__HOST=127.0.0.1
export LUME_RESULTS_DB__PORT=27017
export LUME_RESULTS_DB__USERNAME=root
export LUME_RESULTS_DB__PASSWORD=password
export LUME_RESULTS_DB__DATABASE=test
export LUME_PREFECT__BACKEND=server 
export LUME_BACKEND=docker
export LUME_ENVIRONMENT__LOCAL_PIP_REPOSITORY=...
export LUME_ENVIRONMENT__LOCAL_CONDA_CHANNEL_DIRECTORY=...
```

Start the docker services using:
```
lume-services docker start-services
```

In [1]:
from lume_services.services.models.db import ModelDBConfig
from lume_services.services.results.mongodb import MongodbResultsDBConfig
from lume_services.services.scheduling.backends.server import (
    PrefectAgentConfig,
    PrefectConfig,
    PrefectServerConfig,
)

from lume_services.services.files.filesystems import (
    LocalFilesystem,
    MountedFilesystem,
)
from lume_services.config import LUMEServicesSettings
import logging
logging.basicConfig(level="DEBUG")

model_db_config = ModelDBConfig(
        host="127.0.0.1",
        port="3306",
        user="root",
        password="test",
        database="model_db",
)

results_db_config = MongodbResultsDBConfig(
    port="27017",
    host="127.0.0.1",
    username="root",
    database="test",
    password="password",
)

prefect_config = PrefectConfig(
    server=PrefectServerConfig(
        host="http://localhost", host_port="4200", tag="core-1.2.4"
    ),
    agent=PrefectAgentConfig(host="http://localhost", host_port="5000"),
    backend="server",
)


settings = LUMEServicesSettings(
    model_db=model_db_config,
    results_db=results_db_config,
    prefect=prefect_config,
    backend="docker",
    mounted_filesystem=None,
)

## Use LUME-services config to create injected services

In [2]:
from lume_services import config
config.configure(settings)

# initialize singletons
model_db_service = config.context.model_db_service()
results_db_service = config.context.results_db_service()
scheduling_service = config.context.scheduling_service()

INFO:lume_services.config:Configuring LUME-services environment...
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
INFO:lume_services.config:Environment configured.
DEBUG:lume_services.config:Environment configured using {'model_db': {'host': '127.0.0.1', 'port': 3306, 'user': 'root', 'database': 'model_db', 'connection': {'pool_size': None, 'pool_pre_ping': True}, 'dialect_str': 'mysql+pymysql'}, 'results_db': {'username': 'root', 'host': '127.0.0.1', 'port': 27017, 'authMechanism': 'DEFAULT'}, 'prefect': {'server': {'tag': 'core-1.2.4', 'host': 'http://localhost', 'host_port': '4200', 'host_ip': '127.0.0.1'}, 'ui': {'host': 'http://localhost', 'host_port': '8080', 'host_ip': '127.0.0.1', 'apollo_url': 'http://localhost:4200/graphql'}, 'telemetry': {'enabled': True}, 'agent': {'host': 'http://localhost', 'host_port': '5000'}, 'home_dir': '~/.p

## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Store example model, deployment, flow, and project

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.


In [5]:
# create a project
project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'test' for key 'project.PRIMARY'")
[SQL: INSERT INTO project (project_name, description) VALUES (%(project_name)s, %(description)s)]
[parameters: {'project_name': 'test', 'description': 'my_description'}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
scheduling_service.create_project("test")

## Let's use the source of the model we created in the demo

In [ ]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz"
# populates local channel
model.store_deployment(source_path, project_name="test")

In [ ]:
model.run_and_return(parameters={
                        "input1": "hey", 
                        "input2": "jackie", 
                        "filename": f"/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"}
         )

In [ ]:
model.results()

In [6]:
from lume_services.models import Model

model = Model(model_id=1)

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.


In [7]:
model.load_deployment()

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 ORDER BY deployment.deploy_date DESC
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment_dependencies._id, deployment_dependencies.name, deployment_dependencies.source, deployment_dependencies.local_source, deployment_dependencies.version, deployment_dependencies.deployment_id, deployment_dependencies.dependency_type_id, dependency_type_1.id, dependency_type_1.type 
FROM deployment_dependencies LEFT OUTER JOIN dependency_type AS dependency_type_1 ON dependency_type_1.id = deployment_depend

In [9]:
model.run_and_return(parameters={
                        "input1": "hey", 
                        "input2": "jackie", 
                        "filename": f"/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"
        }
)

INFO:lume_services.services.scheduling.backends.server:Creating Prefect flow run for 7def8c7a-09a0-4449-ade1-ebfecf7c10af with parameters {'input1': 'hey', 'input2': 'jackie', 'filename': '/Users/jgarra/sandbox/lume-services/test_file.txt', 'filesystem_identifier': 'local'} and run_config {"labels": ["lume-services"], "env": {"EXTRA_CONDA_PACKAGES": "_libgcc_mutex libstdcxx-ng ld_impl_linux-64 ca-certificates libgfortran5 libgomp libgfortran-ng _openmp_mutex libgcc-ng lzo xorg-libxdmcp xorg-libxau pthread-stubs libuuid libdeflate lerc jpeg libbrotlicommon libev c-ares libwebp-base ripgrep liblief patchelf patch lz4-c icu libopenblas reproc libiconv keyutils yaml-cpp xz openssl ncurses libzlib libffi bzip2 yaml libnsl libxcb libbrotlienc libbrotlidec libblas reproc-cpp libedit readline zstd libpng libxml2 libssh2 libnghttp2 tk libsqlite libsolv brotli-bin libcblas liblapack krb5 libtiff freetype libarchive sqlite brotli libcurl openjpeg lcms2 hdf5 libmamba tzdata pybind11-abi python whe

DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 3624
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 3624
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 3624
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 36
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 2

RuntimeError: `watch_flow_run` timed out after 0.0 hours of waiting for completion. Your flow run is still in state: <Submitted: "Submitted for execution">